In [ ]:
#### MPC NOTES ####

#Take PV,SP, STATUS from UI + process
#runs continuously
#Comes online in predict mode, need to change status=3 to write to process. Advise do not bring to control mode immediately as it takes 3-10 minutes to get on track.

#Feed, Feed Water, Mill Speed, Sump Level, Cyclone Pressure  =MVs
#Bearing Pressure/Load, Power, Density, D50C  =CVs

# the input function is where values should be input.  the status option could be removed and implemented at a higher level.
# the "Write_to_()" functions should be where values are written to operator interface or to process.

# You’ll want to set up the controller to auto-restart if the computer restarts, it loses communication, or the application crashes. 
#I recommend using the Windows Scheduler to check on the status of the application and attempt to restart it (in WARM mode). It can check every minute or at whatever time you desire. 
#On Linux, use cron to schedule tasks.

# If you are implementing online on a physical system, 
# there is logic in the program that directs the NEWVAL from the controller to the process only when the operator has turned on the controller. 
# This could be changed to be part of the code that controls the Modbus or OPC communication. 
# That layer is another protection against implementing unintended control moves after a controller restart.


In [11]:
# IMPLEMENT MPC ######

######  MPC Grinding Circuit           ######
######  TU 20 Jun 2023                 ######
######  Erik Hobson                    ######
######  Based on ARX w/ na=2 and nb=4  ######
######  MV's: F, FW, MS, SL, CP        ######
######  CV's: BP, P, D, D50            ######

import time
import numpy as np
from gekko import GEKKO

       # model parameter array
p=\
{'a': np.array([[0.78769122, 0.51589105, 0.76237291, 0.09526761],
        [0.14596307, 0.39288084, 0.18718714, 0.08760731]]),
 'b': np.array([[[ 9.98117557e-02,  3.24565166e-02,  1.00533276e+00,
           6.26620574e-02,  1.66080453e+00],
         [ 7.83314704e-02, -7.04742403e-02, -2.20846044e+00,
          -1.00267982e-01, -1.26915560e+00],
         [ 1.91960089e-02, -1.32119749e-01, -7.03888458e-01,
           6.64105040e-02,  1.18589574e-01],
         [ 3.44680719e-02,  4.72906525e-02, -1.11221359e-01,
          -5.34743240e-02,  1.12519536e+00]],
 
        [[-1.91894518e-02,  4.83874705e-01,  2.80772232e+00,
          -4.26214221e-02, -3.52770156e-01],
         [ 6.34705654e-03, -4.83473713e-01,  5.13599568e+00,
          -3.22134904e-03,  1.91555091e-01],
         [ 8.84912719e-03, -1.67281718e-01,  4.23617580e+00,
           3.05919435e-02, -3.53431835e-01],
         [-8.89122104e-03,  2.11342953e-01,  4.26879818e+00,
           2.99758451e-02,  5.69521641e-01]],
 
        [[ 1.22276647e-03,  1.09896065e-02,  2.70743997e-01,
          -4.98825501e-03, -3.71151620e-02],
         [ 2.76772618e-03, -2.39106358e-03,  1.86426784e-02,
           2.00558339e-02, -1.00955867e-01],
         [ 1.67442138e-03, -2.87307940e-03, -2.15592872e-01,
          -3.00218815e-03,  1.73499904e-01],
         [-1.18272054e-03, -7.08093926e-03, -9.46153284e-02,
          -6.92031084e-03, -2.11906425e-02]],
 
        [[ 4.83280010e-07, -4.67322957e-04, -7.58211203e-04,
          -1.63720372e-05, -1.76074253e-03],
         [-7.63978913e-06,  2.86542687e-04, -3.12131098e-04,
           6.32590362e-06, -2.00768905e-04],
         [ 3.20332561e-06,  1.11460524e-04,  3.53089642e-04,
           8.54304195e-07, -3.12011632e-04],
         [-7.11962562e-06,  8.98045672e-05,  8.33563214e-04,
           2.44767138e-06,  3.28703161e-04]]]),
 'c': np.array([ 3.73292946e+02, -5.24589543e+01,  2.24977578e+00,  1.17276828e-01])}

# # MVs initial measurement values
# Fs  = float(input())
# FWs = float(input())
# MSs = float(input())
# SLs = float(input())
# CPs = float(input())
    
# # CV initial measurement values
# BP1m  = float(input())
# BP2m  = float(input())
# BP3m  = float(input())
# BP4m  = float(input())
# BPm= np.mean(BP1m,BP2m,BP3m,BP4m)
# Pm   = float(input())
# Dm   = float(input())
# D50m = float(input())
    
# # CV initial setpoints set as measured values to start
# BPsp = BPm*1.0 
# Psp  = Pm*1.0  
# Dsp  = Dm*1.0  
# D50sp= D50m*1.0

#########################################################
# Initialize Models    
#########################################################
MV = [ 'F', 'FW', 'MS', 'SL', 'CP']
CV = ['BP','P', 'D', 'D50']

#create ARX model:
m = GEKKO(remote=False)
y = m.Array(m.CV,4)
u = m.Array(m.MV,5)
y,u = m.arx(p,y,u)

# BP1=float(input())
# BP2=float(input())
# BP3=float(input())
# BP4=float(input())

# avgBP= (BP1+BP2+BP3+BP4) /4
# y[0].value   = avgBP
# y[1].value   = float(input()) 
# y[2].value   = float(input()) 
# y[3].value   = float(input())

# rename CVs
BP = y[0]; P  = y[1]; D  = y[2]; D50= y[3]
# rename MVs
F  = u[0]; FW = u[1]; MS = u[2]; SL = u[3]; CP = u[4]

# # steady state initialization
m.options.IMODE = 1
m.solve(disp=False)
# set up MPC
m.options.IMODE   = 6 # MPC
m.options.CV_TYPE = 1 # Objective type
m.options.NODES   = 2 # Collocation nodes
m.options.SOLVER  = 3 # 3 IPOPT # can change to 1, APOPT if too many infeasible solutions
m.time=np.linspace(0,30,31) #moving TIME WINDOW for MPC. #Can reduce for faster computation

########################################
#Parameters, limits, and specifications

#deadbands for SPs:
dbd={'BP':100,'P':100,'D':2,'D50':0.005}

#Dictionary of variable limits
lim_dict = {
# Controlled variables
'BP': [5500, 6500],
'P': [2000, 2250],
'D': [50, 60],
'D50': [0.1, 0.2],
# Manipulated variables
'F': [90, 140],
'FW': [10, 30],
'MS': [13, 16],
'SL': [40, 80],
'CP': [8, 14],
}
dmax_dict={'F': 5,'FW': 5,'MS': 1,'SL': 10,'CP': 3}   #this is for 1 min intervals

dcost_dict={'F': 0.0001,'FW': 0.0001,'MS': 0.01,'SL': 0.0001,'CP': 0.0001} #default is 0.00001 #increase for less movement

#tau_dict={}
###########################
for i,mv_i in enumerate(u):
    tag = MV[i]
    mv_i.STATUS = 1     #controlled
    mv_i.FSTATUS = 0    #process: change to 1 
    mv_i.LOWER = lim_dict[tag][0]
    mv_i.UPPER = lim_dict[tag][1]
    mv_i.DMAX = dmax_dict[tag]
    mv_i.DCOST = dcost_dict[tag]

###########################
for i,cv_i in enumerate(y):
    tag = CV[i]
    cv_i.STATUS = 1
    cv_i.FSTATUS = 1   
    cv_i.TAU = 2      #depends on CV: write dict
    cv_i.TR_INIT = 2  #default 0 set 2
    cv_i.TR_OPEN = 2  #ratio of initial to final gap sphi/lo. can change to 0 for ref traj, 1 is default. 2 is ratio of open to final db

#################################################################################################################################################
# Control Loop IMODE= 6 #
#################################################################################################################################################

start_time = time.time()
prev_time = start_time
status=1
cycle=0

try:
    while status!=0:
        
        #READ MV, CV  from process 
        F.MEAS=  float(input())
        FW.MEAS= float(input()) 
        MS.MEAS= float(input()) 
        SL.MEAS= float(input())
        CP.MEAS= float(input()) 
        
        BP1=float(input())
        BP2=float(input())
        BP3=float(input())
        BP4=float(input())
        avgBP= (BP1+BP2+BP3+BP4) /4
        
        BP.MEAS = avgBP
        P.MEAS  = float(input())
        D.MEAS  = float(input())
        D50.MEAS= float(input())
        
        #read SP, ON/WARM/OFF from operator UI
        
        # CV setpoints
        BPsp = float(input())  
        Psp  = float(input())
        Dsp  = float(input())
        D50sp= float(input())

        # Sleep time
        sleep_max = 60.0  # THIS will be 60 seconds in process, but is faster for testing
        sleep = sleep_max - (time.time() - prev_time)
        if sleep>=0.01:
            time.sleep(sleep-0.01)
        else:
            time.sleep(0.01)

        t = time.time()
        dt = t - prev_time
        prev_time = t

        #could Do all these in try/except form
        BP.SPHI = BPsp + dbd['BP'] #read from UI direction
        BP.SPLO = BPsp - dbd['BP']
        P.SPHI = Psp + dbd['P']
        P.SPLO = Psp - dbd['P']
        D.SPHI = Dsp + dbd['D']
        D.SPLO = Dsp - dbd['D']
        D50.SPHI = D50sp + dbd['D50']
        D50.SPLO = D50sp - dbd['D50']

        m.options.MAX_TIME= 57.0
        try:
            m.solve(disp=False)
            m.options.TIME_SHIFT=1
        except:
            m.options.TIME_SHIFT=2
            #display('failed soln')

        #Setting MV targets starting at the next step in time
        #write to UI/process
        print( F.NEWVAL) 
        print(FW.NEWVAL)
        print(MS.NEWVAL)        
        print(SL.NEWVAL)
        print(CP.NEWVAL)
        
        cycle+=1
###############################################################################################################################
except KeyboardInterrupt: #another way to kill controller. can change error!
    status=0
    #print(f'Disconnecting MPC. Runtime={i}')
    
#Note: IF Process or computer shuts down, make controller restart

In [2]:
# #model param array other
# p=\
# {'a': np.array([[0.69240075, 0.33076971, 0.55055693, 0.48697291],
#         [0.23867589, 0.39658054, 0.34590833, 0.39827704]]),
#  'b': np.array([[[ 1.21895901e-01,  2.84086168e-01,  1.45747754e+01,
#            7.29906250e-02,  3.16814067e+00],
#          [ 6.61497020e-02, -6.49371856e-01, -4.14814784e+00,
#           -7.18172979e-02, -2.69357381e+00],
#          [ 6.35881747e-03, -6.29753508e-01, -6.98764002e+00,
#            9.52105722e-02,  1.50631690e+00],
#          [ 3.61675091e-02,  8.64215623e-01, -5.05774294e+00,
#           -1.90783962e-02, -4.47616166e-01]],
 
#         [[-6.81789225e-03, -8.14969081e-02, -2.68313866e+01,
#           -8.03833669e-02, -2.21393946e+00],
#          [-2.53092535e-02,  3.52585202e-01,  1.56509801e+01,
#            7.85260814e-02,  4.60835338e+00],
#          [ 3.92690230e-02,  1.79163134e+00,  3.07491560e+01,
#           -8.28658355e-02, -4.28327393e+00],
#          [-2.11165874e-02, -1.96730454e+00,  2.88659884e+01,
#            2.72935341e-02,  2.08732934e+00]],
 
#         [[ 3.12339667e-03,  5.71372909e-02,  2.47733965e-01,
#            3.26958047e-03, -2.81345035e-01],
#          [ 1.87486446e-03,  1.78121933e-02,  4.91579830e-01,
#            2.09515554e-02, -1.44086880e-01],
#          [ 4.66658450e-03, -4.06104998e-02, -7.16222354e-01,
#           -1.73222868e-02,  5.37162810e-02],
#          [-3.36287202e-04, -3.49049205e-02, -8.69879023e-02,
#           -7.27213898e-03,  4.04838415e-01]],
 
#         [[-8.96353073e-06, -6.36672396e-04,  4.77500551e-03,
#           -8.88634859e-05,  2.45457445e-04],
#          [-1.25741908e-07,  1.22458459e-05, -2.04256141e-03,
#            5.83036193e-05,  3.22155966e-03],
#          [-1.41419838e-05,  4.95883628e-04, -1.09170182e-03,
#            6.03855659e-05, -8.13137084e-04],
#          [ 3.68931155e-06,  1.10404677e-04, -1.34625762e-03,
#            2.64322348e-05, -2.88064425e-03]]]),
#  'c': np.array([ 3.79464753e+02, -1.44072526e+02,  5.27140770e+00,  1.17052589e-02])}

